In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import base64

In [20]:
def get_from_es(body):
    return requests.post(url, json=body, headers={f'Authorization': f'Basic {token}'}).json()

passw = f"{os.getenv('ES_LOGIN_TEDS_BACK')}:{os.getenv('ES_PASSWORD_TEDS_BACK')}"
base64_bytes = passw.encode('ascii')
message_bytes = base64.b64encode(base64_bytes)
token = message_bytes.decode('ascii')
url = os.getenv('ES_URL')+'teds-bibliography/_search'

body = {
    'size': 0,
    'query': {
        'bool': {
            "must_not": [],
            "filter": [
                { "match": { "ipcc.wg_chap.keyword":   "1" }},
            ]
        }
    },
    'aggs': {
        'by_countries': {
            'terms': {
                'field': 'countries.keyword', 'size':10
            },
            # 'aggs': {
            #    'by_is_oa': {
            #         'terms': {'field': 'oa_details.2021Q4.is_oa'},
            #     }
            # }
        }
    }
}
res = get_from_es(body)
res

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'by_countries': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 38615,
   'buckets': [{'key': 'US', 'doc_count': 21135},
    {'key': 'GB', 'doc_count': 11536},
    {'key': 'DE', 'doc_count': 7048},
    {'key': 'AU', 'doc_count': 6681},
    {'key': 'CA', 'doc_count': 4767},
    {'key': 'FR', 'doc_count': 4274},
    {'key': 'CN', 'doc_count': 4202},
    {'key': 'NL', 'doc_count': 4094},
    {'key': 'IT', 'doc_count': 2899},
    {'key': 'CH', 'doc_count': 2871}]}}}

In [19]:
for x in res.get('aggregations').get('by_countries').get('buckets'):
    print(x.get('key'),x.get('doc_count')*100/res.get('hits').get('total').get('value'))

US 63.86
GB 32.62
DE 22.34
FR 16.65
CN 15.53
AU 14.08
CH 11.45
CA 11.32
JP 9.49
NL 9.38
